In [33]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import random
#import gurobipy as gp
#from gurobipy import GRB
import scipy.optimize

In [34]:
#numAsset = 3
#df1 = pd.DataFrame([0.1,0.2,0.3])
#Q = np.mat(np.random.normal(0,1,(3,3)))
#Q = Q*np.transpose(Q)/1000+0.5*np.identity(numAsset)/1000
#print(Q)

#Q = df.cov()

In [35]:
df_exR = pd.read_csv("exR_file.csv")
df_exR
ticker = list(df_exR)[1:] #the first is the time stamp
print(ticker)

['XSP.TO', 'XIU.TO', 'XIN.TO', 'XBB.TO', 'XGB.TO', 'XRB.TO', 'XRE.TO']


In [36]:
df1= df_exR.iloc[:,1:2]
df2 = df_exR.iloc[:,2:3]
df3 = df_exR.iloc[:,3:6]
df4 = df_exR.iloc[:,6:8]
asset_1_index = [0]
asset_2_index = list(range(1,2))
asset_3_index = list(range(2,5))
asset_4_index = list(range(5,7))
print(df1)
print(df2)
asset_2_index
asset_3_index
asset_4_index
#np.array(ticker)[asset_4_index]
type(asset_2_index)

       XSP.TO
0         NaN
1   -0.037262
2    0.002846
3   -0.002867
4    0.017057
..        ...
178  0.027248
179  0.044082
180  0.050627
181  0.004213
182  0.021196

[183 rows x 1 columns]
       XIU.TO
0         NaN
1   -0.035934
2   -0.014420
3    0.018924
4    0.018122
..        ...
178  0.036481
179  0.052970
180  0.023047
181  0.030843
182  0.033390

[183 rows x 1 columns]


list

In [37]:

total_df = pd.concat([df1,df2,df3,df4], axis=1)# merge the 4 asset class
print(total_df)
print(total_df.shape)

print(total_df.shape[1]) # get the amount of columns of the dataframe
numAsset = total_df.shape[1]
print("number of asset is ", numAsset)
Q = total_df.cov()
Q.shape

       XSP.TO    XIU.TO    XIN.TO    XBB.TO    XGB.TO    XRB.TO    XRE.TO
0         NaN       NaN       NaN       NaN       NaN       NaN       NaN
1   -0.037262 -0.035934 -0.059806 -0.000070       NaN -0.013703 -0.002474
2    0.002846 -0.014420  0.007787 -0.020828       NaN -0.021193 -0.009137
3   -0.002867  0.018924 -0.001692  0.028914       NaN  0.040237  0.054973
4    0.017057  0.018122  0.024081  0.014012       NaN  0.015380  0.025324
..        ...       ...       ...       ...       ...       ...       ...
178  0.027248  0.036481  0.027873 -0.027362 -0.031432 -0.042828  0.038720
179  0.044082  0.052970  0.050659 -0.012869 -0.014480 -0.008564  0.047724
180  0.050627  0.023047  0.011021  0.000274  0.001319 -0.020003  0.046757
181  0.004213  0.030843  0.022821  0.005995  0.005625  0.034407  0.026312
182  0.021196  0.033390  0.001299  0.010679  0.011981  0.009638  0.033735

[183 rows x 7 columns]
(183, 7)
7
number of asset is  7


(7, 7)

In [38]:
def formDf(x,ticker):
    x = pd.DataFrame(x).T
    x.columns = ticker
    return x

In [39]:
#Equal Weight strategy
weight_eq = pd.DataFrame(np.repeat(1/numAsset, numAsset))
print(weight_eq)

x_ew = formDf(weight_eq,ticker) # result for equal weight
print(x_ew)

          0
0  0.142857
1  0.142857
2  0.142857
3  0.142857
4  0.142857
5  0.142857
6  0.142857
     XSP.TO    XIU.TO    XIN.TO    XBB.TO    XGB.TO    XRB.TO    XRE.TO
0  0.142857  0.142857  0.142857  0.142857  0.142857  0.142857  0.142857


In [65]:
def rp_obj(y):
    kappa = 5
    return np.matmul(Q,y)-kappa*np.reciprocal(y) #using the gradient of f(y) = 0
def rp_opt(numAsset):
    guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    y = scipy.optimize.fsolve(rp_obj,guess)
    
    x_rp=[]
    y_sum = np.sum(y)
    for i in range(len(y)):
        x_rp.append(y[i]/y_sum)
    x_rp = formDf(x_rp,ticker)
    
    
    return x_rp
x_rp = rp_opt(numAsset)
print(y)
print(type(y))
print(x_rp)



[ 26.74905139  30.19912567  28.12403987  97.04215235 119.80732964
  45.69873152  24.07522027]
<class 'numpy.ndarray'>
     XSP.TO    XIU.TO    XIN.TO   XBB.TO    XGB.TO    XRB.TO    XRE.TO
0  0.071965  0.081247  0.075664  0.26108  0.322326  0.122947  0.064771


In [50]:
# Mean Variance optimization
def meanVarianceOpt(numAsset):
    def objective(weight):#weight
        #print(type(weight))
        #print(weight.shape())
        weight = np.array(weight)
        return weight.dot(Q).dot(weight.T)

    #constraints
    cons = ({'type':"eq",'fun':lambda x: np.sum(x)-1},
           )
    
    #long only
    bounds = tuple((0,1) for x in range(numAsset))
    
    #initialize the weights with an even split, equal weights
    guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    optimized_result = minimize(objective, guess, method = "SLSQP", bounds  = bounds, constraints= cons)
    
    return optimized_result

In [47]:
np.ndarray([1,3,4]).shape

(1, 3, 4)

In [55]:
x_mvo = meanVarianceOpt(numAsset).x
x_mvo = formDf(x_mvo,ticker)

1.0


In [ ]:
# Mean Variance optimization - All weather 

def meanVarianceOpt_allWeather(numAsset):
    def objective(weight):#weight
        weight = np.array(weight)
        return weight.dot(Q).dot(weight.T)

    #constraints
    cons = ({'type':"eq",'fun':lambda x: np.sum(x)-1},
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_1_index])-0.15 },
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_2_index])-0.15 },
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_3_index])-0.15 },
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_4_index])-0.15 },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_1_index]) },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_2_index]) },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_3_index]) },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_4_index]) },
           )
    
    #long only
    bounds = tuple((0,0.3) for x in range(numAsset))
    
    #initialize the weights with an even split, equal weights
    guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    optimized_result = minimize(objective, guess, method = "SLSQP", bounds  = bounds, constraints= cons)
    
    return optimized_result

In [27]:
x_mvo_allWeather = meanVarianceOpt_allWeather(numAsset).x
type(x_mvo_allWeather)
x_mvo_allWeather = formDf(x_mvo_allWeather, ticker)

NameError: name 'meanVarianceOpt_allWeather' is not defined

In [ ]:
x_mvo_allWeather

In [ ]:
np.sum(np.array([1,2,3,4])[[0,2]])

In [ ]:
# Naive Risk Parity

Q

In [ ]:
sigma_sq = np.diagonal(Q)# variance for all assets (sigma^2 )
print(sigma_sq)

In [ ]:
#1/sigma^2
inv_sigma_sq = pd.DataFrame(sigma_sq).rdiv(1).T 
print(inv_sigma_sq)

# 1/sigma
inv_sigma = inv_sigma_sq.apply(np.sqrt)
inv_sigma= np.array(inv_sigma.iloc[0,:].tolist())
print(inv_sigma)
#

In [ ]:
x_naive_rp = []
denom = np.sum(inv_sigma) #sum of 1/sigma at the denominator

for i in range(numAsset):
    x_naive_rp.append((inv_sigma[i]/denom))
print(x_naive_rp)

x_naive_rp = formDf(x_naive_rp, ticker)

In [ ]:
x_naive_rp